In [1]:
import syft as sy
import copy

In [2]:
duet = sy.join_duet("53c69930c0ee494aeb75a9b044f8fd5d")
sy.logger.add(sink="./syft_ds.log")

🎤  🎸  ♪♪♪ Joining Duet ♫♫♫  🎻  🎹

♫♫♫ > DISCLAIMER: Duet is an experimental feature currently in beta.
♫♫♫ > Use at your own risk.


    > ❤️ Love Duet? Please consider supporting our community!
    > https://github.com/sponsors/OpenMined

♫♫♫ > Punching through firewall to OpenGrid Network Node at:
♫♫♫ > http://ec2-18-218-7-180.us-east-2.compute.amazonaws.com:5000
♫♫♫ >
♫♫♫ > ...waiting for response from OpenGrid Network... 
♫♫♫ > DONE!

♫♫♫ > STEP 1: Send the following Duet Client ID to your duet partner!
♫♫♫ > Duet Client ID: 25203aab2bd357dcfcd2556ed5c439ef

♫♫♫ > ...waiting for partner to connect...

♫♫♫ > CONNECTED!


In [3]:
import torch
import torchvision

In [4]:
duet.store.pandas

,ID,Tags,Description,object_type
0,<UID: 6ccc4a72a723483382388a9d0ad9884f>,[x_train],Dataset of 30000 samples,<class 'torch.Tensor'>
1,<UID: d1e7beee877d460da4ce54116876c340>,[y_train],Dataset of 30000 samples,<class 'torch.Tensor'>
2,<UID: f0e2e5e86194469cbeababb5a06ffa88>,[x_test],Dataset of 5000 samples,<class 'torch.Tensor'>
3,<UID: 09043f3804084d3ba45a6c522d041275>,[y_test],Dataset of 5000 samples,<class 'torch.Tensor'>


In [5]:
#cnn 모델을 사용
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.conv1 = self.torch_ref.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = self.torch_ref.nn.Conv2d(32, 64, 3, 1) 
        self.conv3 = self.torch_ref.nn.Conv2d(64, 128, 3 ,1)
        
        self.dropout1 = self.torch_ref.nn.Dropout2d(0.25)
        self.dropout2 = self.torch_ref.nn.Dropout2d(0.5)
        
        self.fc1 = self.torch_ref.nn.Linear(9216, 128)
        self.fc2 = self.torch_ref.nn.Linear(128, 10)

    def forward(self, x):
        # ImgIn shape=(?, 1, 28, 28)
        # Conv     -> (?, 32, 28, 28)
        x = self.conv1(x)
        x = self.torch_ref.nn.functional.relu(x)
        # ImgIn shape=(?, 32, 28, 28)
        # Conv     -> (?, 64, 28, 28)
        # Pool     -> (?, 64, 14, 14)
        x = self.conv2(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.torch_ref.nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        # Flatten     -> (?, 64*14*14)
        x = self.torch_ref.flatten(x, 1)
        x = self.fc1(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = self.torch_ref.nn.functional.log_softmax(x, dim=1)
        return output

In [6]:
local_model = SyNet(torch) #local에 보낼 모델 생성

In [7]:
model = local_model.send(duet) #생성한 모델을 duet을 통해 local에 전송
remote_torch = duet.torch #데이터 오너의 torch사용

In [8]:
# 학습에 사용될 인자들 설정
args = {
    "batch_size": 64,
    "test_batch_size": 5000,
    "epochs": 40,
    "lr": 1.0,
    "gamma": 0.7,
    "dry_run": False,
    "seed": 42, # the meaning of life
    "log_interval": 10,
    "save_model": True,
}

In [9]:
#train 데이터셋 얻기
train_kwargs = {
    "batch_size": args["batch_size"],
}
#duet에 0번째, 1번째가 x_train, y_train
print(len(duet.store[0]), len(duet.store[1]))
#듀엣에 보관중인 데이터는 torch.Tensor형태이므로 
#이 두개를 묶어서 x와 y가 모두 포함된 Dataset을 만들고 
#Dataset을 다시 DataLoader를 만들어서 모델의 입력으로 넣어줄 수 있도록 변환해줌
train_data_ptr=torch.utils.data.TensorDataset(duet.store[0].get_copy(), duet.store[1].get_copy())
train_loader_ptr = torch.utils.data.DataLoader(train_data_ptr,**train_kwargs)
train_data_length = len(train_loader_ptr.dataset)
print(train_data_length)

30000 30000
30000


In [10]:
#test데이터셋 얻기
test_kwargs = {
    "batch_size": args["test_batch_size"],
}
#duet에 2번째, 3번째가 x_test, y_test
print(len(duet.store[2]), len(duet.store[3]))

test_data=torch.utils.data.TensorDataset(duet.store[2].get_copy(), duet.store[3].get_copy())
test_loader = torch.utils.data.DataLoader(test_data, **test_kwargs)
test_data_length = len(test_loader.dataset)
print(test_data_length)

5000 5000
5000


In [11]:
params = model.parameters()
optimizer = remote_torch.optim.Adadelta(params, lr=args["lr"])
scheduler = remote_torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=args["gamma"])

In [12]:
def train(model, torch_ref, train_loader, optimizer, epoch, args, train_data_length):
    train_batches = round((train_data_length / args["batch_size"]) + 0.5) #올림작업을 하기위한 +0.5
    print(f"> Running train in {train_batches} batches")
    if model.is_local:
        print("Training requires remote model")
        return

    model.train() #학습 알림

    for batch_idx, data in enumerate(train_loader):
        data_ptr, target_ptr = data[0], data[1]
        optimizer.zero_grad()
        output = model(data_ptr)
        loss = torch_ref.nn.functional.nll_loss(output, target_ptr)
        loss.backward()
        optimizer.step()
        loss_item = loss.item()
        train_loss = duet.python.Float(0)  #Float
        train_loss += loss_item
        if batch_idx % args["log_interval"] == 0:
            local_loss = None
            local_loss = loss_item.get(
                reason="To evaluate training progress",
                request_block=True,
                timeout_secs=5
            )
            if local_loss is not None:
                print("Train Epoch: {} {} {:.4}".format(epoch, batch_idx, local_loss))
            else:
                print("Train Epoch: {} {} ?".format(epoch, batch_idx))
            if args["dry_run"]:
                break
        if batch_idx >= train_batches - 1:
            print("batch_idx >= train_batches, breaking")
            break

In [13]:
def test_local(model, torch_ref, test_loader, test_data_length):
    # download remote model
    if not model.is_local:
        local_model = model.get(
            request_block=True,
            reason="test evaluation",
            timeout_secs=5
        )
    else:
        local_model = model
    # + 0.5 lets us math.ceil without the import
    test_batches = round((test_data_length / args["test_batch_size"]) + 0.5)
    print(f"> Running test_local in {test_batches} batches")
    local_model.eval()
    test_loss = 0.0
    correct = 0.0

    with torch_ref.no_grad():
        for batch_idx, (data, target) in enumerate(test_loader):
            output = local_model(data)
            iter_loss = torch_ref.nn.functional.nll_loss(output, target, reduction="sum").item()
            test_loss = test_loss + iter_loss
            pred = output.argmax(dim=1)
            total = pred.eq(target).sum().item()
            correct += total
            if args["dry_run"]:
                break
                
            if batch_idx >= test_batches - 1:
                print("batch_idx >= test_batches, breaking")
                break

    accuracy = correct / test_data_length
    print(f"Test Set Accuracy: {100 * accuracy}%")

In [14]:
%%time
import time

args["dry_run"] = False  # comment to do a full train
print("Starting Training")
for epoch in range(1, args["epochs"] + 1):
    epoch_start = time.time()
    print(f"Epoch: {epoch}")
    #로컬 훈련
    train(model, remote_torch, train_loader_ptr, optimizer, epoch, args, train_data_length)
    #로컬 테스트
    test_local(model, torch, test_loader, test_data_length)
    scheduler.step()
    epoch_end = time.time()
    print(f"Epoch time: {int(epoch_end - epoch_start)} seconds")
    break
print("Finished Training")

Starting Training
Epoch: 1
> Running train in 469 batches
Train Epoch: 1 0 2.309
Train Epoch: 1 10 2.004
Train Epoch: 1 20 1.181
Train Epoch: 1 30 0.5613
Train Epoch: 1 40 0.6126
Train Epoch: 1 50 0.4565
Train Epoch: 1 60 0.5415
Train Epoch: 1 70 0.3528
Train Epoch: 1 80 0.3757
Train Epoch: 1 90 0.7534
Train Epoch: 1 100 0.3522
Train Epoch: 1 110 0.2441
Train Epoch: 1 120 0.2844
Train Epoch: 1 130 0.3704
Train Epoch: 1 140 0.3851
Train Epoch: 1 150 0.2367
Train Epoch: 1 160 0.2693
Train Epoch: 1 170 0.1395
Train Epoch: 1 180 0.1822
Train Epoch: 1 190 0.1782
Train Epoch: 1 200 0.4341
Train Epoch: 1 210 0.08853
Train Epoch: 1 220 0.1022
Train Epoch: 1 230 0.03211
Train Epoch: 1 240 0.2129
Train Epoch: 1 250 0.1511
Train Epoch: 1 260 0.2294
Train Epoch: 1 270 0.223
Train Epoch: 1 280 0.3243
Train Epoch: 1 290 0.2877
Train Epoch: 1 300 0.1102
Train Epoch: 1 310 0.1707
Train Epoch: 1 320 0.097
Train Epoch: 1 330 0.1867
Train Epoch: 1 340 0.1699
Train Epoch: 1 350 0.1226
Train Epoch: 1 360 0